In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [2]:
symbols, comp_names, market_caps, stock_prices, revenues = [], [], [], [], []

In [3]:
# Crawling the stock symbols from Nasdaq
driver = webdriver.Chrome()
url = 'https://stockanalysis.com/list/nasdaq-stocks/'
actions = ActionChains(driver)
driver.get(url)
time.sleep(2)
count = 0

while True:
    if count == 7:
        break
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tbody')))

    # Get the list of stock elements
    stocks_container = driver.find_element(By.TAG_NAME, 'tbody')
    stocks = stocks_container.find_elements(By.TAG_NAME, 'tr')

    # Iterate through each stock element
    for stock in stocks:
        try:
            # There is only one a in each stock. By finding it I will get the stock symbol
            stock_symbol = stock.find_element(By.TAG_NAME, 'a').text
            symbols.append(stock_symbol)
        except:
            print("Symbol not found")

        try:
            # Get the company name
            comp_name = stock.find_element(By.CSS_SELECTOR, '.slw.svelte-eurwtr').text
            comp_names.append(comp_name)
        except:
            print("Company name not found")

        try:
            # Get the market cap and stock price
            elements = stock.find_elements(By.CSS_SELECTOR, '.svelte-eurwtr')
            single_class_elements = [elem for elem in elements if len(elem.get_attribute("class").split()) == 1]
            market_cap = single_class_elements[1].text
            stock_price = single_class_elements[2].text
            
            if market_cap == '-':
                market_caps.append(np.nan)
            else:
                market_caps.append(market_cap)
            stock_prices.append(stock_price)
        except:
            print("Market cap or stock price not found")

        try:
            # Get the revenue
            revenue = stock.find_element(By.CSS_SELECTOR, '.tr.svelte-eurwtr').text
            revenues.append(revenue)
        except:
            print("Revenue not found")

    # Try to find and click the "Next" button
    try:
        next_page_btn = driver.find_element (
            By.CSS_SELECTOR,
            'button.controls-btn.xs\\:pl-1.xs\\:pr-1\\.5.bp\\:text-sm.sm\\:pl-3.sm\\:pr-1'
        )
        if 'disabled' in next_page_btn.get_attribute('class'):
            print("No more pages to scrape.")
            break
        actions.move_to_element(next_page_btn).click().perform()
        count += 1
        time.sleep(2)  # Wait for the new page to load
    except:
        print("Next page button not found or no more pages.")
        break
    
driver.quit()

In [13]:
#Saving the initial data to a csv file

data = {
    'Symbol': symbols,
    'Company Name': comp_names,
    'Market Cap': market_caps,
    'Stock Price': stock_prices,
    'Revenue': revenues
}

df = pd.DataFrame(data)

df.to_csv('nasdaq_stocks.csv', index=False)